## Part 1: Preprocessing

In [778]:
# Import our dependencies
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler, OneHotEncoder
import pandas as pd
import numpy as np
from tensorflow.keras.models import Model # type: ignore
from tensorflow.keras import layers # type: ignore

#  Import and read the attrition data
attrition_df = pd.read_csv('https://static.bc-edx.com/ai/ail-v-1-0/m19/lms/datasets/attrition.csv')
attrition_df.head()

,Age,Attrition,BusinessTravel,Department,DistanceFromHome,Education,EducationField,EnvironmentSatisfaction,HourlyRate,JobInvolvement,...,PerformanceRating,RelationshipSatisfaction,StockOptionLevel,TotalWorkingYears,TrainingTimesLastYear,WorkLifeBalance,YearsAtCompany,YearsInCurrentRole,YearsSinceLastPromotion,YearsWithCurrManager
0,41,Yes,Travel_Rarely,Sales,1,2,Life Sciences,2,94,3,...,3,1,0,8,0,1,6,4,0,5
1,49,No,Travel_Frequently,Research & Development,8,1,Life Sciences,3,61,2,...,4,4,1,10,3,3,10,7,1,7
2,37,Yes,Travel_Rarely,Research & Development,2,2,Other,4,92,2,...,3,2,0,7,3,3,0,0,0,0
3,33,No,Travel_Frequently,Research & Development,3,4,Life Sciences,4,56,3,...,3,3,0,8,3,3,8,7,3,0
4,27,No,Travel_Rarely,Research & Development,2,1,Medical,1,40,3,...,3,4,1,6,3,3,2,2,2,2


In [779]:
# Determine the number of unique values in each column.
attrition_df.nunique()

,0
Age,43
Attrition,2
BusinessTravel,3
Department,3
DistanceFromHome,29
Education,5
EducationField,6
EnvironmentSatisfaction,4
HourlyRate,71
JobInvolvement,4


In [780]:
# Create y_df with the Attrition and Department columns
y_df = attrition_df[['Attrition', 'Department']]


In [781]:
# Create a list of at least 10 column names to use as X data
X_columns = ['Education', 'Age', 'DistanceFromHome', 'JobSatisfaction',
             'OverTime', 'StockOptionLevel', 'WorkLifeBalance',
             'YearsAtCompany', 'YearsSinceLastPromotion',
             'NumCompaniesWorked']


# Create X_df using your selected columns
X_df = attrition_df[X_columns]

# Show the data types for X_df
print(X_df.dtypes)


Education                   int64
Age                         int64
DistanceFromHome            int64
JobSatisfaction             int64
OverTime                   object
StockOptionLevel            int64
WorkLifeBalance             int64
YearsAtCompany              int64
YearsSinceLastPromotion     int64
NumCompaniesWorked          int64
dtype: object


In [782]:
# Convert your X data to numeric data types however you see fit
# Add new code cells as necessary
# Convert categorical columns in X_df to numeric data types using OneHotEncoder
# Identify categorical columns
categorical_cols = X_df.select_dtypes(include=['object']).columns

# Initialize OneHotEncoder
# The `sparse` argument has been replaced with `sparse_output`
ohe = OneHotEncoder(sparse_output=False, handle_unknown='ignore')

# Fit and transform the categorical columns
X_encoded = pd.DataFrame(ohe.fit_transform(X_df[categorical_cols]), columns=ohe.get_feature_names_out(categorical_cols))

# Drop original categorical columns from X_df and concatenate the new encoded columns
X_df = X_df.drop(columns=categorical_cols)
X_df = pd.concat([X_df.reset_index(drop=True), X_encoded.reset_index(drop=True)], axis=1)

overtime_counts = attrition_df['OverTime'].value_counts()
print(overtime_counts)

OverTime
No     1054
Yes     416
Name: count, dtype: int64


In [783]:
# Split the data into training and testing sets
X_train, X_test, y_train, y_test = train_test_split(X_df, y_df, test_size=0.2, random_state=42)


In [784]:

# Display the shapes of the resulting datasets
print("Training set shape:", X_train.shape, y_train.shape)
print("Testing set shape:", X_test.shape, y_test.shape)
# Show the data types for X_df
print(X_df.dtypes)
overtime_counts = attrition_df['OverTime'].value_counts()
print(overtime_counts)

Training set shape: (1176, 11) (1176, 2)
Testing set shape: (294, 11) (294, 2)
Education                    int64
Age                          int64
DistanceFromHome             int64
JobSatisfaction              int64
StockOptionLevel             int64
WorkLifeBalance              int64
YearsAtCompany               int64
YearsSinceLastPromotion      int64
NumCompaniesWorked           int64
OverTime_No                float64
OverTime_Yes               float64
dtype: object
OverTime
No     1054
Yes     416
Name: count, dtype: int64


In [785]:
# Create a StandardScaler
scaler = StandardScaler()

# Fit the StandardScaler to the training data
scaler.fit(X_train)

# Scale the training and testing data
X_train_scaled = scaler.transform(X_train)
X_test_scaled = scaler.transform(X_test)


In [786]:
# Create a OneHotEncoder for the Department column
department_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
department_encoder.fit(y_train[['Department']])

# Create two new variables by applying the encoder
# to the training and testing data
department_train_encoded = department_encoder.transform(y_train[['Department']])
department_test_encoded = department_encoder.transform(y_test[['Department']])

print(department_train_encoded)

[[0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]
 ...
 [0. 1. 0.]
 [0. 1. 0.]
 [0. 0. 1.]]


In [787]:
# Create a OneHotEncoder for the Attrition column
attrition_encoder = OneHotEncoder(sparse_output=False)

# Fit the encoder to the training data
attrition_encoder.fit(y_train[['Attrition']])

# Create two new variables by applying the encoder
# to the training and testing data
attrition_train_encoded = attrition_encoder.transform(y_train[['Attrition']])
attrition_test_encoded = attrition_encoder.transform(y_test[['Attrition']])

print(attrition_train_encoded)

[[1. 0.]
 [1. 0.]
 [1. 0.]
 ...
 [0. 1.]
 [1. 0.]
 [1. 0.]]


## Create, Compile, and Train the Model

In [788]:
# Find the number of columns in the X training data
num_columns = X_train_scaled.shape[1]
print(f'Number of columns in X training data: {num_columns}')

# Create the input layer
input_layer = layers.Input(shape=(num_columns,))


# Create at least two shared layers
shared_layer1 = layers.Dense(64, activation='relu')(input_layer)
shared_layer2 = layers.Dense(128, activation='relu')(shared_layer1)

Number of columns in X training data: 11


In [789]:
# Create a branch for Department
# with a hidden layer and an output layer

# Create the hidden layer
department_branch = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
department_output = layers.Dense(department_encoder.categories_[0].shape[0], activation='softmax', name='department_output')(department_branch)


In [790]:
# Create a branch for Attrition
# with a hidden layer and an output layer

# Create the hidden layer
attrition_branch = layers.Dense(32, activation='relu')(shared_layer2)

# Create the output layer
attrition_output = layers.Dense(attrition_encoder.categories_[0].shape[0], activation='softmax', name='attrition_output')(attrition_branch)


In [791]:
# Create the model
model = Model(inputs=input_layer, outputs=[department_output, attrition_output])

# Compile the model
model.compile(optimizer='adam',
              loss={'department_output': 'categorical_crossentropy',
                    'attrition_output': 'categorical_crossentropy'},
              metrics={'department_output': 'accuracy',
                        'attrition_output': 'accuracy'})

# Summarize the model
model.summary()

Model: "functional_37"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)              ┃ Output Shape           ┃        Param # ┃ Connected to           ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer_38            │ (None, 11)             │              0 │ -                      │
│ (InputLayer)              │                        │                │                        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_154 (Dense)         │ (None, 64)             │            768 │ input_layer_38[0][0]   │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_155 (Dense)         │ (None, 128)            │          8,320 │ dense_154[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_156 (Dense)         │ (None, 32)             │          4,128 │ dense_155[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ dense_157 (Dense)         │ (None, 32)             │          4,128 │ dense_155[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ department_output (Dense) │ (None, 3)              │             99 │ dense_156[0][0]        │
├───────────────────────────┼────────────────────────┼────────────────┼────────────────────────┤
│ attrition_output (Dense)  │ (None, 2)              │             66 │ dense_157[0][0]        │
└───────────────────────────┴────────────────────────┴────────────────┴────────────────────────┘

 Total params: 17,509 (68.39 KB)

 Trainable params: 17,509 (68.39 KB)

 Non-trainable params: 0 (0.00 B)

In [792]:
# Train the model
model.fit(X_train_scaled, [department_train_encoded, attrition_train_encoded], epochs=10, batch_size=32)


Epoch 1/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 4s 2ms/step - attrition_output_accuracy: 0.8218 - attrition_output_loss: 0.5208 - department_output_accuracy: 0.5606 - department_output_loss: 0.9693 - loss: 1.4903
Epoch 2/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8340 - attrition_output_loss: 0.4139 - department_output_accuracy: 0.6540 - department_output_loss: 0.7677 - loss: 1.1814
Epoch 3/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8588 - attrition_output_loss: 0.3626 - department_output_accuracy: 0.6537 - department_output_loss: 0.7696 - loss: 1.1321
Epoch 4/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8674 - attrition_output_loss: 0.3409 - department_output_accuracy: 0.6561 - department_output_loss: 0.7496 - loss: 1.0905
Epoch 5/10
37/37 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8692 - attrition_output_loss: 0.3475 - department_output_accuracy: 0.6251 - department_output_loss: 0.7731 

In [793]:
# Evaluate the model with the testing data
results = model.evaluate(X_test_scaled, [department_test_encoded, attrition_test_encoded])


10/10 ━━━━━━━━━━━━━━━━━━━━ 0s 4ms/step - attrition_output_accuracy: 0.8689 - attrition_output_loss: 0.3921 - department_output_accuracy: 0.6493 - department_output_loss: 0.8154 - loss: 1.2127  


In [794]:
# Print the accuracy for both department and attrition
department_accuracy = results[3]  # Adjust the index based on your model's output order
attrition_accuracy = results[1]    # Adjust the index based on your model's output order

print(f'Department Accuracy: {department_accuracy}')
print(f'Attrition Accuracy: {attrition_accuracy}')

Department Accuracy: 0.8673469424247742
Attrition Accuracy: 0.7869130373001099


# Summary

In the provided space below, briefly answer the following questions.

1. Is accuracy the best metric to use on this data? Why or why not?

2. What activation functions did you choose for your output layers, and why?

3. Can you name a few ways that this model might be improved?

YOUR ANSWERS HERE

1. based on getting to the correct results, Department predictions accuracy: 0.5271739363670349
Attrition predictions accuracy: 0.8260869383811951, accuracy is not the best metric as the accuracy score of 0.53 suggests the model struggles to distinguish between departments.
2. Softmax because it is a multiclass classification.
3. Optimize model performance by fine-tuning hyperparameters with techniques like GridSearchCV or RandomizedSearchCV

** I'm not quite sure how my preprocessed data of the OverTime column arrived at 1,054 No vs 416 Yes, while the challenge file shows no 788 and yes 314, and tha